# **STEMfit example**

This notebook gives a short demonstration of how to detect atoms in a STEM image and determine the unit cell. First, we start by importing `STEMfit` and `Plots`:

In [ ]:
using Pkg; Pkg.activate("X:\\Documents\\virtualenvs\\Julia_STEMfit");
#sing Pkg; Pkg.activate("C:\\Users\\ewout\\Documents\\venv\\Julia-STEMfit");

#using Pkg; Pkg.activate("C:\\Users\\Ewout\\Documents\\venv\\Julia\\STEMfit");
using Revise
import STEMfit
#using Plots

Now, we load our example image. The image we use here is of three oxide perovskite layers grown epitaxially on each other. By setting `convert=true`, we let `STEMfit` convert our 16 bit TIFF image into the required format.

In [ ]:
image = STEMfit.load_image("image.tif", convert=true);

We filter the raw image using a combination of singular value decomposition and Gaussian convolution. `STEMfit` will automatically determine the number of singular vectors to use. For diagnostic purposes, we can set the keyword argument `plot=true`:

In [ ]:
filtered_image = STEMfit.filter_image(image, plot=false);

Then, we use the `find_atoms` function to find atoms in the image using an adaptive thresholding procedure. The function takes the image as an argument as well as `bias` and `window_size` parameters which control the thresholding. Increasing `bias` tends to make the threshold higher (i.e. more separation of atoms), `window_size` determines how big of an area is considered at each point in the image. The function returns a matrix of atom positions, a vector of atom widths, a vector of atom intensities and the thresholded image. The latter can be used to assess the quality of the thresholding and asjust the `bias` and `window_size` parameters. 

In [ ]:
(atom_parameters, thresholded_image) = STEMfit.find_atoms(filtered_image, bias=0.5, window_size=3, min_atom_size=10);

All atoms are correctly detected. `STEMfit` will now try to find potential unit cells in the atomic positions. To do this, it finds clusters in the vectors between each pair of atoms using the [DBSCAN](https://www.wikiwand.com/en/DBSCAN) algorithm. Any two of such vectors may form the side of a possible unit cell. We restrict the angle of the unit cell to 85-95 degrees (i.e. a rectangular unit cell) using the `uc_allowed_angles` parameter and use the default settings for all other parameters. The possible choices for unit cells are automatically plotted together with the average *local* atomic environment.

In [ ]:
(unit_cells, neighbors, atom_tree) = STEMfit.find_unit_cells(atom_parameters, uc_allowed_angles=85:95);

It seems like `STEMfit` found a correct unit cell (Unit cell 3). Bear in mind that this unit cell represents the *average* translational symmetry in the whole image. To transform our atomic positions (in pixel units) to positions in length units (e.g. nm), we can use part of the image as a reference. In this case, the bottom ~1/5 of the image is a substrate with known lattice parameters. First, we calculate the local lattice parameter for each atom. We use the `calculate_lattice_parameters` function using the average unit cell we found and otherwise standard settings. 

In [ ]:
lattice_parameters = STEMfit.calculate_lattice_parameters(atom_parameters, unit_cells[3])

Valid lattice parameters cannot be calculated for some atoms, because they do not have enough neighbors to get accurate results. `STEMfit` can give us a filter of only the valid atoms:

In [ ]:
valid_atoms = STEMfit.valid_lattice_parameter_filter(lattice_parameters);

We select the local lattice parameter values of only those atoms with a y-index larger that 80% of the height of the image as the reference:

In [ ]:
#Select only atoms in the bottom 20% of the image
substrate_min_y = 0.8*size(image)[1]
reference_lattice_parameters = lattice_parameters[:, atom_parameters[1,:] .> substrate_min_y .&& valid_atoms]

Since the reference in this image is SrTiO<sub>3</sub>, we know that each basis vector corresponds to a distance of 0.3905 nm. We can use this fact to transform out atomic positions into length units. We ask the `get_pixel_size` function to return the in-plane and out-of-plane pixel sizes separately by setting `return_two_sizes=true`, because there is a relatively large difference between the two likely due to drift in the image. 

In [ ]:
pixel_sizes = STEMfit.get_pixel_size(reference_lattice_parameters, (0.3905, 0.3905))

We can now transform our atomic positions and local lattice parameter values into length units:

In [ ]:
atom_parameters_in_nm = STEMfit.convert_to_nm(atom_parameters, pixel_sizes)
lattice_parameters_in_nm = STEMfit.convert_to_nm(lattice_parameters, pixel_sizes)

Let's inspect the results using a histogram. The lattice parameters we calculated above are defined along the two basis vectors of the unit cell. In this case, those correspond to the in-plane/horizontal and the out-of-plane/vertical direction, respectively. However, this depends on the choice of unit cell (for example, had we chosen Unit cell 5, the basis vectors would be rotated 30<sup>o</sup> with respect to the horizontal and vertical directions). 

In [ ]:
lp_histogram = STEMfit.plot_histogram(lattice_parameters_in_nm, xlabel="Lattice parameter (nm)");

As we can see, basis vector 1 (in-plane) is smaller on average than basis vector 2 (out-of-plane), which we expect to see for a compressively strained epitaxial film such as the present example. Furthermore, three distinct peaks are visible in the out-of-plane (basis vector 2) direction, corresponding to the three layers of the film. The in-plane direction shows only a single peak, indicating little to no strain relaxation. Finally, we plot the results as a lattice parameter map. We filter out the atoms for which no accurate lattice parameter information could be calculated.

In [ ]:
(lp_map_1, lp_map_2) = STEMfit.map_lattice_parameter(atom_parameters_in_nm, lattice_parameters_in_nm);

Finally, we would like to convert our lattice parameter into strain. To do this, we have to divide the local lattice parameter by the bulk lattice parameter for each layer. Hence, we need to assign to each atom a layer index manually. To help with this, we can show the image with a grid, then choose the y positions of each of the interfaces.

In [ ]:
STEMfit.plot_image_with_grid!(image)

It seems like the layer 1/layer 2 interface is at y = 550 and the layer 2/layer 3 interface at y = 970. We can now divide the atom positions into the three layers and assign to them the indices 1, 2 and 3, respectively: 

In [ ]:
layer_assignment = STEMfit.layer_assignments(atom_parameters, [550, 970]);

Now, we provide the bulk lattice parameters along basis vectors 1 and 2 of the unit cell (which in this case correspond to the in-plane and out-of plane lattice parameters).

In [ ]:
bulk_lattice_parameters = Dict(
                               1=>(0.404, 0.399), 
                               2=>(0.395355, 0.395355), 
                               3=>(0.3905, 0.3905)
                               );

Now calculate the strain from the lattice parameters and plot in the same we we plotted the lattice parameters before.

In [ ]:
strain = STEMfit.get_strain_from_lattice_parameters(lattice_parameters_in_nm, bulk_lattice_parameters, layer_assignment)

In [ ]:
strain_histogram = STEMfit.plot_histogram(strain, xlabel="Strain");

In [ ]:
(strain_map_1, strain_map_2) = STEMfit.map_strain(atom_parameters_in_nm[:, valid_atoms], strain[:, valid_atoms]);

Finally, we can save our results and the generated plots:

In [ ]:
STEMfit.save_atomic_positions("example results\\results.csv", 
                                atom_parameters=atom_parameters_in_nm, 
                                lattice_parameters=lattice_parameters_in_nm, 
                                strain=strain, 
                                valid_atoms=valid_atoms)

In [ ]:
STEMfit.savefig(lp_histogram, "example results\\lattice parameter histogram.png")
STEMfit.savefig(strain_histogram, "example results\\strain histogram.png")

STEMfit.savefig(lp_map_1, "example results\\lattice parameter map 1.png")
STEMfit.savefig(lp_map_2, "example results\\lattice parameter map 2.png")

STEMfit.savefig(strain_map_1, "example results\\strain map 1.png")
STEMfit.savefig(strain_map_2, "example results\\strain map 2.png");

# Gaussian fitting

First, we determine the background contribution:

In [ ]:
background_image = STEMfit.construct_background(filtered_image, unit_cells[3]);

We then obtain initial parameters for the gaussians using the atomic parameters we calculated using the center of mass mode:

In [ ]:
fitting_parameters = STEMfit.fitting_parameters(atom_parameters, background_image);

Finally, we fit the model to the image. This can take a few minutes. 

In [ ]:
image_model = STEMfit.ImageModel(fitting_parameters, unit_cells[3], background_image);

In [ ]:
STEMfit.fit!(image_model, image)

Let's look at the results. On the left, we display the experimental data, on the right is the fitted model. 

In [ ]:
STEMfit.show_images(image, STEMfit.produce_image(image_model))

An excellent fit has been achieved, so we save the results to a file:

In [ ]:
STEMfit.save_model(image_model, "example results\\results_fitted.csv")